In [20]:
# multi headed multi-step cnn
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt

from query1 import *
from Func_Paper1 import *

In [21]:
df = data1(30)

df.index = df['datetime']
df.index = df.index.astype('datetime64[ns]')
df.sort_index(inplace=True)
df.dropna(inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])
print("shape ", df.shape)

shape  (30, 33)


In [22]:
df.tail(1)

,datetime,tail24,tail24_avg,evap,infl,losses,rel1,rel2,rel3,rel_tol,...,run_c2,run_c3,spillway,irr,camp,demand,derate,outage,stor,fore24
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-04,2020-06-04,84.55,82.42,0.5644,1.743,0.0,3.7706,4.2132,5.272,13.2558,...,85.1,100.3,0.0,0.0,11508.0,13.0,75.0,0.0,4034.8776,139.66


In [23]:
dtat0 = df.iloc[-1:,0:1].values[0][0]
dti = pd.date_range(dtat0, periods=30).shift(1, freq='D')
dfdt = dti.to_frame(index=False, name='datims')

In [24]:
# dfdt

In [25]:
df.drop(('datetime'), axis=1, inplace=True)

In [26]:
from sklearn.preprocessing import MinMaxScaler
y_scaler = MinMaxScaler()
y_scaler.fit(df[['fore24']])
X_scaler = MinMaxScaler()

df[['tail24', 'tail24_avg', 'evap', 'infl', 'losses',
       'rel1', 'rel2', 'rel3', 'rel_tol', 'engr1', 'engr2', 'engr3', 'cond1',
       'cond2', 'cond3', 'str1', 'str2', 'str3', 'run_g1', 'run_g2', 'run_g3',
       'run_c1', 'run_c2', 'run_c3', 'spillway', 'irr', 'camp', 'demand',
       'derate', 'outage', 'stor','fore24']] = X_scaler.fit_transform(df)

dfval = df.values[:,:-1]

In [27]:
def split_dataset(data):
    test = data
    test = array(split(test, len(test)/30))
    return test

In [28]:
test = split_dataset(df.values)

In [29]:
print('test', test.shape)

test (1, 30, 32)


In [30]:
n_input = 14

In [31]:
def forecast(model, history, n_input):

    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

    input_x = data[-n_input:, :]

    input_x = [input_x[:,i].reshape((1,input_x.shape[0],1)) for i in range(input_x.shape[1])]

    yhat = model.predict(input_x, verbose=1)

    yhat = yhat[0]
    
    return yhat

In [32]:
#เป็น list ที่สมาชิกแต่ละตัวเป็น samples (windows, n-feature)
history = [x for x in test]

In [33]:
print(len(history))
print(type(history))
print((array(history).shape))
print(type(array(history)))

1
<class 'list'>
(1, 30, 32)
<class 'numpy.ndarray'>


In [34]:
model = ShallowConvNet(test, n_input)
# model.summary()
model.load_weights("models/weights73.hdf5")
model.compile(loss='mse', optimizer='adam')

In [35]:
predictions = list()
for i in range(len(test)):
    # predict the week
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
#     history.append(test[i, :])
# evaluate predictions days for each week
predictions = array(predictions)

1/1 [==============================] - 0s 309ms/step


In [36]:
yp = y_scaler.inverse_transform(predictions)

In [37]:
from sqlalchemy import create_engine
user='root'
pwd='opt#565784'
dbname='predictive'
host='10.211.1.25'
port=3306
con = create_engine(f'mysql+pymysql://{user}:{pwd}@{host}:{port}/{dbname}')

df = pd.DataFrame(yp).T
df.columns = ['stor30D']
com = pd.concat([dfdt, df], axis=1, join='inner')
com.to_sql('monthly_m', con, if_exists='replace', index=False)